In [2]:
from SBox import *

In [3]:
# XOR-profile

def gen_xor_profile(self):
    xor_profile = [[0 for _ in range(2**self.output_len)] for _ in range(2**self.input_len)]
    for input1 in range(2**self.input_len):
        for input2 in range(2**self.input_len):
            output1 = self.sbox[input1]
            output2 = self.sbox[input2]
            input_xor = input1 ^ input2
            output_xor = output1 ^ output2
            xor_profile[input_xor][output_xor] += 1
    return xor_profile

SBox.gen_xor_profile = gen_xor_profile

aes_profile = aes_sbox.gen_xor_profile()

for i in range(2**aes_sbox.input_len):
    print(aes_profile[i])

print([box.gen_xor_profile() for box in des_sboxes])

testbox = SBox([1,2,7,4,5,6,3,0], 3, 3)

print(testbox.gen_xor_profile())

[256, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 4, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 

In [4]:
# Single round AES

from aes import *

round_keys = expand_key(key)

def aes_inner(plaintext):
    plaintext = bytes.fromhex(hex(plaintext)[2:].ljust(32, '0'))
    state = bytes2matrix(plaintext)
    state = sub_bytes(state)
    state = shift_rows(state)
    state = mix_columns(state)
    byte = matrix2bytes(state)
    return int.from_bytes(byte, 'big')

def aes_round(key, plaintext):
    plaintext = bytes.fromhex(hex(plaintext)[2:].ljust(32, '0'))

    # Convert plaintext to state matrix
    state = bytes2matrix(plaintext)

    # Perform one inner round of AES with some key
    state = add_round_key(state, round_keys[0])
    state = sub_bytes(state)
    state = shift_rows(state)
    state = mix_columns(state)
    state = add_round_key(state, round_keys[1])

    # Convert state matrix to ciphertext
    ciphertext = matrix2bytes(state)

    return ciphertext

key = b"\xc3,\\\xa6\xb5\x80^\x0c\xdb\x8d\xa5z*\xb6\xfe\\"

key0 = matrix2bytes(round_keys[0])
key1 = matrix2bytes(round_keys[1])

xor_profile = aes_sbox.xor_profile

plausible_keys = {}

input_diffs = {}
output_diffs = {}

for i in range(2**aes_sbox.input_len):
    i = int.from_bytes(i.to_bytes(1, 'big'), 'big')
    input_diffs[i] = []
    output_diffs[i] = []

def gen_pairs():
    for input1 in range(2**aes_sbox.input_len):
        for input2 in range(2**aes_sbox.input_len):
            output1 = aes_inner(input1)
            output2 = aes_inner(input2)
            input_xor = input1 ^ input2
            output_xor = output1 ^ output2
            input_diffs[input_xor] += [(input1, input2)]
            output_diffs[output_xor] += [(output1, output2)]

gen_pairs()

# for input_diff in range(2**aes_sbox.input_len):
#     for input1 in range(2**aes_sbox.input_len):
#         input2 = input1 ^ input_diff
#         output1 = aes_round(key, input1)
#         output2 = aes_round(key, input2)
#         output_diff = int.from_bytes(output1, 'big') ^ int.from_bytes(output2, 'big')




b'crypto{MYAES128}'
b'\xd1O\x14j\xa4+O\xb6\xa1\xc4\x08B)\x8f\x12\xdd'
b'\xd1O\x14j\xa4+O\xb6\xa1\xc4\x08B)\x8f\x12\xdd'


AttributeError: 'SBox' object has no attribute 'xor_profile'